<a href="https://colab.research.google.com/github/fralfaro/MAT281/blob/main/docs/labs/lab_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°04


**Objetivo**: Aplicar técnicas intermedias y avanzadas de análisis de datos con pandas utilizando un caso real: el Índice de Libertad de Prensa. Este laboratorio incluye operaciones de limpieza, transformación, combinación de datos, y análisis exploratorio usando `merge`, `groupby`, `concat` y otras funciones fundamentales.




**Descripción del Dataset**

El presente conjunto de datos está orientado al análisis del **Índice de Libertad de Prensa**, una métrica internacional que evalúa el nivel de libertad del que gozan periodistas y medios de comunicación en distintos países. Este índice es recopilado anualmente por la organización **Reporteros sin Fronteras**.

La base de datos contempla observaciones por país y año, e incluye tanto el valor del índice como el ranking correspondiente. A menor puntaje en el índice, mayor nivel de libertad de prensa.

**Diccionario de variables**

| Variable     | Clase    | Descripción                                                                          |
| ------------ | -------- | ------------------------------------------------------------------------------------ |
| `codigo_iso` | carácter | Código ISO 3166-1 alfa-3 que representa a cada país.                                 |
| `pais`       | carácter | Nombre oficial del país.                                                             |
| `anio`       | entero   | Año en que se registró la medición del índice.                                       |
| `indice`     | numérico | Valor numérico del Índice de Libertad de Prensa (menor valor indica mayor libertad). |
| `ranking`    | entero   | Posición relativa del país en el ranking mundial de libertad de prensa.              |


**Fuente original y adaptación pedagógica**

* **Fuente original**: [Reporteros sin Fronteras](https://www.rsf-es.org/), recopilado y publicado a través del portal del [Banco Mundial](https://tcdata360.worldbank.org/indicators/h3f86901f?country=BRA&indicator=32416&viz=line_chart&years=2001,2019).
* **Adaptación educativa**: Los archivos han sido modificados intencionalmente para incorporar desafíos técnicos que permiten aplicar los contenidos abordados en clases, tales como limpieza de datos, normalización, detección de duplicados, y combinación de fuentes.


**Descripción de los archivos disponibles**

* **`libertad_prensa_codigo.csv`**: Contiene los pares `codigo_iso` y `pais`. Incluye intencionalmente un código ISO con dos nombres distintos de país para efectos de limpieza y validación de datos.

* **`libertad_prensa_01.csv`**: Contiene registros de los años **anteriores a 2010**. Incluye las variables `PAIS`, `ANIO`, `INDICE`, y `RANKING` con nombres de columna en **mayúsculas**.

* **`libertad_prensa_02.csv`**: Contiene registros de los años **desde 2010 en adelante**. Estructura similar al archivo anterior, con nombres de columna también en **mayúsculas**.





In [2]:
import numpy as np
import pandas as pd

# lectura de datos
archivos_anio = [
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_01.csv',
    'https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_02.csv'
 ]
df_codigos = pd.read_csv('https://raw.githubusercontent.com/fralfaro/MAT281/main/docs/labs/data/libertad_prensa_codigo.csv')



### 1. Consolidación y limpieza de datos

A partir de los archivos disponibles, realice los siguientes pasos:

**a)** Cree un DataFrame llamado `df_anio` que consolide la información proveniente de los archivos **`libertad_prensa_01.csv`** y **`libertad_prensa_02.csv`**, correspondientes a distintas ventanas de tiempo. Recuerde que ambos archivos tienen nombres de columnas en mayúscula, por lo que debe normalizarlas a **minúscula** para asegurar consistencia.

**b)** Explore el archivo **`libertad_prensa_codigo.csv`** e identifique el código ISO que aparece asociado a dos nombres de país distintos. Elimine el registro que corresponda a un valor incorrecto o inconsistente, conservando solo el que considere válido.

**c)** Una vez preparados los archivos, cree un nuevo DataFrame llamado `df` que combine `df_anio` con `df_codigos`, utilizando la columna `codigo_iso` como clave. Asegúrese de realizar una unión que conserve únicamente los registros que tengan coincidencia en ambas fuentes.

> **Sugerencia**:
>
> * Para unir los archivos por filas (años), utilice la función `pd.concat([...])`.
> * Para combinar información por columnas (variables), utilice `pd.merge(...)` especificando `on='codigo_iso'`.



In [3]:
"a)"
lp1=pd.read_csv(archivos_anio[0])
lp2=pd.read_csv(archivos_anio[1])

lp1.columns = lp1.columns.str.lower()
lp2.columns = lp2.columns.str.lower()
df_anio = pd.concat([lp1, lp2], ignore_index=True)
"b)"
df_codigos[df_codigos.duplicated('codigo_iso', keep=False)]
df_codigos = df_codigos.drop_duplicates('codigo_iso', keep='first')
"c)"
df = pd.merge(df_anio, df_codigos, on='codigo_iso', how='inner')
df.head()

,codigo_iso,anio,indice,ranking,pais
0,AFG,2001,35.5,59.0,Afghanistán
1,AGO,2001,30.2,50.0,Angola
2,ALB,2001,NaN,NaN,Albania
3,AND,2001,NaN,NaN,Andorra
4,ARE,2001,NaN,NaN,Emiratos Árabes Unidos




### 2. Exploración inicial del conjunto de datos

Una vez que hayas consolidado el DataFrame final `df`, realiza un análisis exploratorio básico respondiendo las siguientes preguntas:

#### **Estructura del DataFrame**

* ¿Cuántas **filas (observaciones)** contiene el conjunto de datos?
* ¿Cuántas **columnas** tiene el DataFrame?
* ¿Cuáles son los **nombres de las columnas**?
* ¿Qué **tipo de datos** tiene cada columna?
* ¿Hay columnas con un tipo de dato inesperado (por ejemplo, fechas como strings)?

#### **Resumen estadístico**

* Genera un resumen estadístico del conjunto de datos con `.describe()`.
  ¿Qué observas sobre los valores de `indice` y `ranking`?
* ¿Qué valores mínimo, máximo y promedio tiene la columna `indice`?
* ¿Qué países presentan los valores extremos en `indice` y `ranking`?

#### **Datos faltantes**

* ¿Cuántos valores nulos hay en cada columna?
* ¿Qué proporción de observaciones tienen valores faltantes?
* ¿Hay columnas con más del 30% de datos faltantes?

#### **Unicidad y duplicados**

* ¿Cuántos países distintos (`pais`) hay en el DataFrame?
* ¿Cuántos años distintos (`anio`) hay representados?
* ¿Existen filas duplicadas (exactamente iguales)? ¿Cuántas?

#### **Validación cruzada de columnas**

* ¿Hay inconsistencias entre el país (`pais`) y su código (`codigo_iso`)?
  (por ejemplo, un mismo código ISO asociado a más de un país)

> **Sugerencia**: Apoya tu análisis con funciones como `.info()`, `.nunique()`, `.isnull().sum()`, `.duplicated()`, `.value_counts()`, entre otras.



    

In [4]:
# FIXME"Estructura del DataFrame"
print(f"Hay {df.shape[0]} Filas (observaciones)")
print(f"Hay {df.shape[1]} Columnas (variables)")
print(f"El nombre de las columnas son: {list(df.columns)}")
print("Los tipos de datos por cada columna son:")
print(df.dtypes)
print("No hay columnas con un tipo de dato inesperado")

"Resumen estadístico"
display(df.describe())
print(f"El mínimo es {df['indice'].min()}")
print(f"El máximo es {df['indice'].max()}")
print(f"El promedio es {df['indice'].mean():.2f}")
print(f"La mediana es {df['indice'].median()}")

min_indice = df[df['indice'] == df['indice'].min()]
max_indice = df[df['indice'] == df['indice'].max()]
min_ranking = df[df['ranking'] == df['ranking'].min()]
max_ranking = df[df['ranking'] == df['ranking'].max()]
print(f"El pais con menor índice es{min_indice['pais'].values[0]} ({min_indice['anio'].values[0]}) y con mayor índice es {max_indice['pais'].values[0]} ({max_indice['anio'].values[0]})")
print(f"Mientras que, el que tiene menor ranking es {min_ranking['pais'].values[0]} ({min_ranking['anio'].values[0]}) y mayor ranking es {max_ranking['pais'].values[0]} ({max_ranking['anio'].values[0]})")

"Datos faltantes"
if df.isnull().sum().sum() == 1:
  print(f"Hay {df.isnull().sum().sum()} valor nulo en el DataFrame")
elif df.isnull().sum().sum() > 1:
  print(f"Hay {df.isnull().sum().sum()} valores nulos en el DataFrame")
else:
  print(f"No hay valores nulos en el DataFrame")
print("La proporción de valores nulos por columna es")
proporcion_nulos = (df.isnull().sum() / len(df)) * 100
display(proporcion_nulos.round(2))
columnas_problematicas = proporcion_nulos[proporcion_nulos > 30]
print("Las columnas con más del 30% de datos faltantes son:")
display(columnas_problematicas)

"Unicidad y duplicados"
print(f"Hay {df['pais'].nunique()} países distintos")
print(f"Hay {df['anio'].nunique()} años distintos")
print(f"Hay {df.duplicated().sum()} filas duplicadas")

"Validación cruzada de columnas"
inconsistencias = df.groupby('codigo_iso')['pais'].nunique()
print("No se encuentran inconsistencias. Luego, la información general del documento es")
display(df.info())
print(f"Las primeras 5 filas son:")
print(df.head())
print(f"Las últimas 5 filas son:")
print(df.tail())
print(f"Distribución de observaciones por año:")
display(df['anio'].value_counts().sort_index())

Hay 3060 Filas (observaciones)
Hay 5 Columnas (variables)
El nombre de las columnas son: ['codigo_iso', 'anio', 'indice', 'ranking', 'pais']
Los tipos de datos por cada columna son:
codigo_iso     object
anio            int64
indice        float64
ranking       float64
pais           object
dtype: object
No hay columnas con un tipo de dato inesperado


,anio,indice,ranking
count,3060.000000,2664.000000,2837.000000
mean,2009.941176,205.782316,477.930913
std,5.786024,2695.525264,6474.935347
min,2001.000000,0.000000,1.000000
25%,2005.000000,15.295000,34.000000
50%,2009.000000,28.000000,70.000000
75%,2015.000000,41.227500,110.000000
max,2019.000000,64536.000000,121056.000000


El mínimo es 0.0
El máximo es 64536.0
El promedio es 205.78
La mediana es 28.0
El pais con menor índice esDinamarca (2008) y con mayor índice es Kosovo (2014)
Mientras que, el que tiene menor ranking es Finlandia (2001) y mayor ranking es Kosovo (2015)
Hay 619 valores nulos en el DataFrame
La proporción de valores nulos por columna es


,0
codigo_iso,0.00
anio,0.00
indice,12.94
ranking,7.29
pais,0.00


Las columnas con más del 30% de datos faltantes son:


,0


Hay 179 países distintos
Hay 17 años distintos
Hay 0 filas duplicadas
No se encuentran inconsistencias. Luego, la información general del documento es
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3060 entries, 0 to 3059
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   codigo_iso  3060 non-null   object 
 1   anio        3060 non-null   int64  
 2   indice      2664 non-null   float64
 3   ranking     2837 non-null   float64
 4   pais        3060 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 119.7+ KB


None

Las primeras 5 filas son:
  codigo_iso  anio  indice  ranking                    pais
0        AFG  2001    35.5     59.0             Afghanistán
1        AGO  2001    30.2     50.0                  Angola
2        ALB  2001     NaN      NaN                 Albania
3        AND  2001     NaN      NaN                 Andorra
4        ARE  2001     NaN      NaN  Emiratos Árabes Unidos
Las últimas 5 filas son:
     codigo_iso  anio  indice  ranking       pais
3055        WSM  2019   18.25     22.0      Samoa
3056        YEM  2019   61.66    168.0      Yemen
3057        ZAF  2019   22.19     31.0  Sudáfrica
3058        ZMB  2019   36.38    119.0     Zambia
3059        ZWE  2019   42.23    127.0   Zimbabue
Distribución de observaciones por año:


,count
anio,
2001,180
2002,180
2003,180
2004,180
2005,180
2006,180
2007,180
2008,180
2009,180





### 3. Comparación regional: países latinoamericanos

En esta sección se busca identificar cuáles son los países de América Latina que han presentado los valores extremos del **Índice de Libertad de Prensa** en cada año observado.

> Recuerda que un menor puntaje en `indice` implica mayor libertad de prensa.

#### **Tareas:**

**a)** Utilizando un ciclo `for`, recorre cada año del conjunto de datos filtrado por países latinoamericanos, y determina para cada año:

* El país con el menor valor de `indice` (mayor libertad de prensa).
* El país con el mayor valor de `indice` (menor libertad de prensa).

**b)** Resuelve la misma tarea del punto anterior utilizando un enfoque vectorizado con `groupby`, sin usar ciclos explícitos.



#### **Lista de países latinoamericanos considerada:**

```python
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
           'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
           'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
           'USA', 'VEN']
```

> Puedes usar esta lista para filtrar el DataFrame final por la columna `codigo_iso`.



In [8]:
# respuesta
america = ['ARG', 'ATG', 'BLZ', 'BOL', 'BRA', 'CAN', 'CHL', 'COL', 'CRI',
       'CUB', 'DOM', 'ECU', 'GRD', 'GTM', 'GUY', 'HND', 'HTI', 'JAM',
       'MEX', 'NIC', 'PAN', 'PER', 'PRY', 'SLV', 'SUR', 'TTO', 'URY',
       'USA', 'VEN']

df_america = df[df['codigo_iso'].isin(america)].copy()

"a)"
resultados_for = []
for anio in df_america['anio'].unique():
    df_anual = df_america[df_america['anio'] == anio].dropna(subset=['indice'])
    if not df_anual.empty:
        min_idx = df_anual['indice'].idxmin()
        max_idx = df_anual['indice'].idxmax()

        if pd.notna(min_idx) and pd.notna(max_idx):
            mejor_pais = df_anual.loc[min_idx]
            peor_pais = df_anual.loc[max_idx]
            resultados_for.append({
                'anio': anio,
                'mejor_pais': mejor_pais['pais'],
                'mejor_indice': mejor_pais['indice'],
                'peor_pais': peor_pais['pais'],
                'peor_indice': peor_pais['indice']
            })
df_resultados_for = pd.DataFrame(resultados_for)
display(df_resultados_for)


"b)"
df_america_filtrado = df_america.dropna(subset=['indice'])

vec = df_america_filtrado.groupby('anio')['indice'].agg(['min', 'max']).reset_index()
vec = vec.rename(columns={'min': 'mejor_indice', 'max': 'peor_indice'})

mejores_paises = df_america_filtrado.loc[df_america_filtrado.groupby('anio')['indice'].idxmin()][['anio', 'pais', 'indice']].rename(columns={'pais': 'mejor_pais', 'indice': 'mejor_indice'})
peores_paises =df_america_filtrado.loc[df_america_filtrado.groupby('anio')['indice'].idxmax()][['anio', 'pais', 'indice']].rename(columns={'pais': 'peor_pais', 'indice': 'peor_indice'})

df_vec = pd.merge(mejores_paises, peores_paises, on='anio')

display(vec)

,anio,mejor_pais,mejor_indice,peor_pais,peor_indice
0,2001,Canadá,0.80,Cuba,90.30
1,2002,Trinidad y Tobago,1.00,Cuba,97.83
2,2003,Trinidad y Tobago,2.00,Argentina,35826.00
3,2004,Trinidad y Tobago,2.00,Cuba,87.00
4,2005,Bolivia,4.50,Cuba,95.00
5,2006,Canadá,4.88,Cuba,96.17
6,2007,Canadá,3.33,Cuba,88.33
7,2008,Canadá,3.70,Cuba,94.00
8,2009,Estados Unidos,6.75,Cuba,78.00
9,2012,Jamaica,9.88,Cuba,71.64


,anio,mejor_indice,peor_indice
0,2001,0.80,90.30
1,2002,1.00,97.83
2,2003,2.00,35826.00
3,2004,2.00,87.00
4,2005,4.50,95.00
5,2006,4.88,96.17
6,2007,3.33,88.33
7,2008,3.70,94.00
8,2009,6.75,78.00
9,2012,9.88,71.64


### 4. Análisis anual del índice por país

En esta sección se busca analizar la evolución del **índice máximo** de libertad de prensa alcanzado por cada país a lo largo del tiempo.

#### **Tarea principal:**

* Construye una tabla dinámica (`pivot_table`) donde las **filas** correspondan a los países, las **columnas** a los años (`anio`) y los **valores** sean el `indice` máximo alcanzado por cada país en ese año.
* Asegúrate de reemplazar los valores nulos resultantes con `0`.

> **Hint**: Puedes utilizar el parámetro `fill_value=0` en `pd.pivot_table(...)`.



#### **Preguntas adicionales:**

**a)** ¿Qué país tiene el mayor valor de `indice` en toda la tabla resultante? ¿Y cuál tiene el menor (distinto de cero)?
**b)** ¿Qué años presentan en promedio los valores de `indice` más altos? ¿Y los más bajos?

> (Pista: usa `.mean(axis=0)` sobre la tabla pivot)

**c)** ¿Qué país muestra mayor **variabilidad** (diferencia entre su máximo y mínimo `indice` a lo largo del tiempo)?

> (Pista: aplica `.max(axis=1) - .min(axis=1)`)

**d)** ¿Existen países con índice constante a lo largo de todos los años registrados? ¿Cuáles?

**e)** ¿Qué países no tienen ningún dato (es decir, quedaron con todos los valores igual a 0)? ¿Podrías explicar por qué?





In [17]:
pivot_table = pd.pivot_table(df, values='indice', index='pais', columns='anio', fill_value=0)
display(pivot_table)

"a)"
max_g = pivot_table[pivot_table > 0].max().max()
min_g_nonulo = pivot_table[pivot_table > 0].min().min()
pais_max_val = pivot_table[pivot_table == max_g].stack().idxmax()[0]
pais_min_val = pivot_table[pivot_table == min_g_nonulo].stack().idxmin()[0]
print(f"País con el mayor índice: {pais_max_val} ({max_g})")
print(f"País con el menor índice: {pais_min_val} ({min_g_nonulo})")
media_anual = pivot_table.mean(axis=0).sort_values()
print(f"Años con el índice promedio más bajo: {media_anual.index[:5].tolist()}")
print(f"Años con el índice promedio más alto: {media_anual.index[-5:].tolist()}")

"b)"
variability_by_country = pivot_table.max(axis=1) - pivot_table.min(axis=1)
pais_mayor_variabilidad = variability_by_country.idxmax()
print(f"País con mayor variabilidad: {pais_mayor_variabilidad} ({variability_by_country.max()})")

"c)"
paises_indice_constante = pivot_table[(pivot_table.nunique(axis=1) == 1) & (pivot_table.iloc[:, 0] != 0)].index.tolist()
print(f"Países con índice constante (distinto de 0): {paises_indice_constante}")

"d)"
paises_sin_datos = pivot_table[(pivot_table == 0).all(axis=1)].index.tolist()
print(f"Países sin datos: {paises_sin_datos}")

anio,2001,2002,2003,2004,2005,2006,2007,2008,2009,2012,2013,2014,2015,2017,2018,2019
pais,,,,,,,,,,,,,,,,
Afghanistán,35.5,40.17,28.25,39.17,44.25,56.50,59.25,54.25,51.67,37.36,37.07,37.44,37.75,39.46,37.28,36.55
Albania,0.0,6.50,11.50,14.17,18.00,25.50,16.00,21.75,21.50,30.88,29.92,28.77,29.92,29.92,29.49,29.84
Alemania,1.5,1.33,2.00,4.00,5.50,5.75,4.50,3.50,4.25,10.24,10.23,11.47,14.80,14.97,14.39,14.60
Algeria,31.0,33.00,43.50,40.33,40.00,40.50,31.33,49.56,47.33,36.54,36.26,36.63,41.69,42.83,43.13,45.75
Andorra,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,6.82,6.82,19.87,19.87,21.03,22.21,24.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,81.3,89.17,86.88,73.25,67.25,79.25,86.17,81.67,75.75,71.78,72.36,72.63,74.27,73.96,75.05,74.93
West Bank y Gaza,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,42.90,42.96,44.68
Yemen,34.8,41.83,48.00,46.25,54.00,56.67,59.00,83.38,82.13,69.22,67.26,66.36,67.07,65.80,62.23,61.66


País con el mayor índice: Kosovo (64536.0)
País con el menor índice: Austria (0.5)
Años con el índice promedio más bajo: [2001, 2002, 2018, 2019, 2017]
Años con el índice promedio más alto: [2009, 2014, 2015, 2012, 2013]
País con mayor variabilidad: Kosovo (64536.0)
Países con índice constante (distinto de 0): []
Países sin datos: []
